# Selecting Assets

In [2]:
from statsmodels.regression.rolling import RollingOLS
from statsmodels.tsa.stattools import coint
from statsmodels.tsa.stattools import adfuller
import seaborn as sn
import matplotlib.pyplot as plt
import statsmodels.api as sm
import datetime as dt
import yfinance as yf
import pandas as pd
import numpy as np
from scipy import stats
import websocket
import json
import time
from datetime import datetime
import sys

#Alpaca imports
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce

import os
os.environ['APCA_API_BASE_URL'] = 'https://paper-api.alpaca.markets'



#Storage
order_history = pd.DataFrame()
spread_visualisations = {}
zscore_visualisations = {}

plt.style.use('fivethirtyeight')

#Setting dataset for the model
END_DATE = dt.date.today()
START_DATE = END_DATE-pd.DateOffset(days=365)

SYMBOLS_LIST = ['XOM', 'CVX', 'BP', 'COP', 'PXD', 'EOG', 'APA', 'OXY', 'MPC', 'SLB', 'HAL', 'KMI', 'PBR', 'SU', 'ENB', 'EPD', 'EQT', 'BHP', 'VALE', 'RIO', 'FCX', 'NEM', 'GOLD', 'WPM', 'AGI', 'TECK', 'SBSW', 'CF', 'ADM', 'MOS', 'FMC', 'LIN', 'NUE', 'RGLD', 'WY', 'PAA', 'ET', 'MPLX', 'WMB', 'X', 'AA', 'CMP', 'IP', 'PKG', 'PPG']




prices_df = yf.download(tickers=SYMBOLS_LIST, start=START_DATE, end=END_DATE)['Adj Close']
prices_df = prices_df.dropna(axis=0)

#Finding the highest correlation pairs
corr_matrix = prices_df.corr().abs()
cmu = corr_matrix.unstack()
cmu = cmu[cmu != 1]
cmu = cmu[~cmu.duplicated()]
cmu = cmu.sort_values(kind="quicksort", ascending=False)
cmu = cmu.reset_index()
cmu = cmu.rename(columns={"level_0" : 'Stock_1', "level_1": 'Stock_2', 0: 'Correlation'})


highest_corr_pairs = cmu[cmu['Correlation'] >= 0.80]
highest_corr_pairs['lookup'] = highest_corr_pairs['Stock_1'] + ' - '+ highest_corr_pairs['Stock_2']
highest_corr_pairs
prices_df

[*********************100%%**********************]  45 of 45 completed


C:\Users\aidan\AppData\Local\Temp\ipykernel_25244\3078593692.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  highest_corr_pairs['lookup'] = highest_corr_pairs['Stock_1'] + ' - '+ highest_corr_pairs['Stock_2']


,AA,ADM,AGI,APA,BHP,BP,CF,CMP,COP,CVX,...,SBSW,SLB,SU,TECK,VALE,WMB,WPM,WY,X,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2022-12-05,45.972157,88.716331,9.697228,44.608044,60.771145,33.462864,101.367111,43.906422,114.420479,169.885056,...,10.321188,50.477436,29.586174,35.489571,15.093271,32.335499,38.377697,29.949677,26.068661,103.322357
2022-12-06,47.741077,88.794525,9.637613,42.812397,61.024036,32.860535,99.427536,43.080414,110.816376,165.507065,...,10.321188,50.157051,28.557089,36.019119,15.368363,31.673084,38.308601,29.474588,26.991533,100.450401
2022-12-07,46.051216,91.062035,9.886005,42.607464,60.741966,32.420734,98.467537,43.660587,110.116646,165.997772,...,10.454611,49.092396,28.376049,35.597439,14.827353,31.975899,38.910717,30.111208,26.445749,100.228004
2022-12-08,46.723202,90.309464,9.975427,41.738918,61.734070,32.439857,98.095306,42.683777,107.931183,166.979218,...,10.530851,49.210693,28.490391,36.538860,15.065763,31.539843,39.009422,30.092203,25.909885,100.972588
2022-12-09,44.549114,89.801216,9.776713,40.850857,62.230118,32.229519,96.684708,42.279236,105.458160,161.648682,...,10.483201,46.302608,28.204535,35.979893,15.533418,31.233164,38.486275,29.911669,25.215250,100.121635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-27,26.299999,73.190002,14.060000,36.340000,61.310001,35.939999,75.900002,24.469999,114.800003,144.360001,...,4.460000,52.549999,31.887510,35.619999,14.890000,36.389999,47.630001,31.430481,34.820000,103.959999
2023-11-28,26.080000,74.050003,14.570000,36.380001,61.639999,35.939999,75.599998,24.510000,114.980003,145.509995,...,4.660000,52.310001,32.381432,36.740002,15.090000,36.330002,49.150002,31.350962,35.419998,103.900002
2023-11-29,26.440001,73.190002,14.600000,36.310001,60.799999,36.070000,74.750000,24.660000,113.970001,143.910004,...,4.530000,51.869999,32.578999,37.080002,14.940000,36.439999,49.049999,31.510002,35.580002,102.339996


In [3]:
def connect_to_alpaca():
    global trading_client
    
    api_key = ''
    api_secret = ''
    
    trading_client = TradingClient(api_key, api_secret, paper=True)
    account = trading_client.get_account()
    print('Connected to ALPACA')
    print('${} is available as buying power.'.format(account.buying_power))

In [4]:
def find_cointegrated_pairs(df, p_value_thresh):
        n = len(df.columns)
        pairs = {}
        for i in range(n):
            for j in range(i+1, n):
                S1 = df.iloc[:,i]
                S2 = df.iloc[:,j]
                result = coint(S1, S2, trend="c", autolag="BIC")
                pvalue = round(result[1], 5)
                if pvalue <= p_value_thresh:
                    pairs[f"{df.columns[i]} - {df.columns[j]}"] = pvalue
        return pairs

#Filtering for pairs with cointegration below threshold(0.05)
pairs = find_cointegrated_pairs(df=prices_df, p_value_thresh=0.05)
pairs_df = pd.DataFrame.from_dict(pairs, orient='index')
pairs_df = pairs_df.rename(columns={0: 'Cointegration'})
pairs_df
# #Mergin with df of highest correlation pairs
highest_corr_pairs = highest_corr_pairs.merge(pairs_df, left_on='lookup', right_on=pairs_df.index)
highest_corr_pairs

,Stock_1,Stock_2,Correlation,lookup,Cointegration
0,RIO,VALE,0.927950,RIO - VALE,0.01232
1,EOG,HAL,0.896468,EOG - HAL,0.03054
2,PAA,SBSW,0.887692,PAA - SBSW,0.03867
3,EQT,PBR,0.887039,EQT - PBR,0.02913
4,BHP,VALE,0.885658,BHP - VALE,0.03070
5,EOG,SLB,0.874528,EOG - SLB,0.02248
6,LIN,SBSW,0.869504,LIN - SBSW,0.03290
7,ENB,NEM,0.854799,ENB - NEM,0.01119
8,SU,X,0.852963,SU - X,0.00111
9,AA,LIN,0.848111,AA - LIN,0.03372


In [5]:
def downloaded_stockdata(stock_1, stock_2):
    # Downloading the required data
    selected_df = yf.download(tickers=[stock_1, stock_2],
                              start=START_DATE,
                              end=dt.date.today() + dt.timedelta(days=1))
# #     #Adding the real latest price
#     selected_df.loc[selected_df.index[-1], ('Adj Close', stock_1)] =  get_last_price((bybit_string_conversion(stock_1)))
#     selected_df.loc[selected_df.index[-1], ('Adj Close', stock_2)] =  get_last_price((bybit_string_conversion(stock_2)))

    selected_df = selected_df.stack()

    
#     #Finding the required metrics
    selected_df['return'] = (selected_df['Adj Close']-selected_df['Open'])/selected_df['Open']
    selected_df['forward_return'] = selected_df.groupby(level=1)['return'].transform(lambda x: x.shift(-1))
    selected_df = selected_df[['Adj Close', 'forward_return']].unstack().droplevel(axis=1, level=0)
    

    selected_df.columns = [stock_1, stock_2, f'{stock_1}_forward_return', f'{stock_2}_forward_return']
    selected_df[f'{stock_1}_return'] = np.log(selected_df[stock_1]).diff()
    selected_df[f'{stock_2}_return'] = np.log(selected_df[stock_2]).diff()
    selected_df['hedge_ratio'] = RollingOLS(selected_df[f'{stock_2}_return'],
                                            selected_df[f'{stock_1}_return'],
                                            window=60).fit().params.values
    selected_df['spread'] = (selected_df[stock_1]-selected_df[stock_2]*selected_df['hedge_ratio'])
    
    #Correlation test
    selected_df['roll_corr'] = selected_df[stock_1].rolling(180).corr(selected_df[stock_2])
    
    # ZScore
    def smooth_zscore(spread):
        return (spread.rolling(1).mean()-spread.rolling(50).mean())/spread.rolling(50).std()
    
    selected_df['zscored'] = smooth_zscore(selected_df['spread'])
    
    #Trading Signal
    selected_df['signal'] = selected_df.apply(lambda x: 1 if (x['zscored']<-1)
                                              else (-1 if (x['zscored']>1) else 0), axis=1)
    
    return selected_df

In [6]:
def adf_test(stock_1, stock_2):
    removed_na_df = downloaded_stockdata(stock_1, stock_2).dropna()
    adf_result = adfuller(removed_na_df['spread'])[1]

    if adf_result <= 0.05:
        return ('P-Value: ' + str(round(adf_result, 2)) + ' - Passed')
    else:
        return ('P-Value: ' + str(round(adf_result, 2)) + ' - Failed')

# Running ADF test
adf_list = []
for n in range(len(highest_corr_pairs)):
    result = adf_test(highest_corr_pairs['Stock_1'][n], highest_corr_pairs['Stock_2'][n])
    adf_list.append(result)

highest_corr_pairs['adf_test'] = adf_list
highest_corr_pairs

[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  2 of 2 completed
[*********************100%%**********************]  2 of 2 completed


,Stock_1,Stock_2,Correlation,lookup,Cointegration,adf_test
0,RIO,VALE,0.927950,RIO - VALE,0.01232,P-Value: 0.93 - Failed
1,EOG,HAL,0.896468,EOG - HAL,0.03054,P-Value: 0.16 - Failed
2,PAA,SBSW,0.887692,PAA - SBSW,0.03867,P-Value: 0.5 - Failed
3,EQT,PBR,0.887039,EQT - PBR,0.02913,P-Value: 0.83 - Failed
4,BHP,VALE,0.885658,BHP - VALE,0.03070,P-Value: 0.95 - Failed
5,EOG,SLB,0.874528,EOG - SLB,0.02248,P-Value: 0.06 - Failed
6,LIN,SBSW,0.869504,LIN - SBSW,0.03290,P-Value: 0.93 - Failed
7,ENB,NEM,0.854799,ENB - NEM,0.01119,P-Value: 0.26 - Failed
8,SU,X,0.852963,SU - X,0.00111,P-Value: 0.54 - Failed
9,AA,LIN,0.848111,AA - LIN,0.03372,P-Value: 0.3 - Failed


In [7]:
def gather_data(stock_1, stock_2):
    global last_day, last_signal, hr, selected_df
    #Assign selected_df  and last_day variable
    selected_df = downloaded_stockdata(stock_1, stock_2)
    print('Collected historical data')
    
    last_day = selected_df.tail(1) 
    last_signal = last_day.signal.values[0]
    hr = round(last_day.hedge_ratio.values[0], 2)
    
    print('Analysed signals')


In [8]:
def pause_algo():
    for remaining in range(5*60*60, 0, -1):
        sys.stdout.write("\r")
        sys.stdout.write("Paused Algorithm ")
        sys.stdout.write("{:2d} seconds remaining.".format(remaining))
        sys.stdout.flush()
        time.sleep(1)

In [9]:
def enter_long_hedge_position(stock_1, stock_2, leverage):
    
    global trading_client, hr

    #Placing Buy order of 1
    market_order = trading_client.submit_order(
                order_data=MarketOrderRequest(
                    symbol=stock_1,
                    qty=1*leverage,
                    side=OrderSide.BUY,
                    time_in_force=TimeInForce.DAY
                    ))
    print(stock_1 +' LONG order executed')

    #Placing Short Order of Hedge Ratio Dot
    market_order_2 = trading_client.submit_order(
                order_data=MarketOrderRequest(
                    symbol=stock_2,
                    qty=round(hr*leverage),
                    side=OrderSide.SELL,
                    time_in_force=TimeInForce.DAY
                    ))
    print(stock_2+' SHORT order executed')

In [10]:
def enter_short_hedge_position(stock_1, stock_2, leverage):
    global trading_client, hr

    #Placing Buy order of 1 AAVE
    
    market_order = trading_client.submit_order(
            order_data=MarketOrderRequest(
                symbol=stock_1,
                qty=1*leverage,
                side=OrderSide.SELL,
                time_in_force=TimeInForce.DAY
                ))
    print(stock_1 +' SHORT order executed')

    #Placing Short Order of Hedge Ratio Dot
    market_order = trading_client.submit_order(
            order_data=MarketOrderRequest(
                symbol=stock_1,
                qty=hr*leverage,
                side=OrderSide.BUY,
                time_in_force=TimeInForce.DAY
                ))
    
    print(stock_2+' LONG order executed')


In [11]:
def positions():
    assets = pd.DataFrame()
    if in_position():
        for n in range(len(trading_client.get_all_positions())):
            pos = dict(trading_client.get_all_positions()[n])
            pos = pd.DataFrame.from_dict(pos, orient='index').T
            assets = pd.concat([assets, pos])
            
            #Changin columns from str to float type
            columns_to_convert = ['unrealized_pl', 'cost_basis', 'market_value',
                                  'avg_entry_price', 'qty','unrealized_plpc' ]
            for column in columns_to_convert:
                assets[column] = assets[column].astype(float)    
    return assets

def unrealised_profit(stock_1, stock_2):
    filtered_pos = positions()[positions()['symbol'].isin([stock_1, stock_2])]
    
    unrealised_profit = filtered_pos['unrealized_pl'].sum()
    cost_basis = filtered_pos['cost_basis'].sum()
    percent_profit = unrealised_profit/cost_basis
    
    return unrealised_profit, percent_profit


In [12]:
def take_profit(stock):
    
    side = str(positions().query("symbol == @stock")['side'][0])
    qty = str(positions().query("symbol == @stock")['qty'][0])
    
    def extract_position_side(string):
        return string.split('.')[-1]
    
    if side == 'PositionSide.LONG':
            market_order = trading_client.submit_order(
                    order_data=MarketOrderRequest(
                        symbol=stock,
                        qty=qty,
                        side=OrderSide.SELL,
                        time_in_force=TimeInForce.DAY
                        ))
            print('Sold ' + qty + ' shares of ' + stock + ' to exit ' + extract_position_side(side) + ' position')
            
    elif side == 'PositionSide.SHORT':
        market_order = trading_client.submit_order(
                order_data=MarketOrderRequest(
                    symbol=stock,
                    qty=qty,
                    side=OrderSide.BUY,
                    time_in_force=TimeInForce.DAY
                    ))
        print('Longed ' + qty + ' shares of ' + stock + ' to exit ' + extract_position_side(side) + ' position')


In [13]:
def in_position():
    global trading_client
    return bool(trading_client.get_all_positions())

In [14]:
def execute_algo(stock_1, stock_2, leverage):#ENSURE TO ADD CORRECT ASSETS HERE
    #defining global variables
    global trading_client, last_day, last_signal, hr
    
    sys.stdout.write('\n')
    print('Assets chosen: ' + stock_1 + ' and ' + stock_2)
    print('Leverage: ' + str(leverage))
    
    #Connect to exchange
    connect_to_alpaca()
    
    while not in_position():
        #Collect the data 
        gather_data(stock_1, stock_2)
        #Trade execution
        if last_signal == 1:
            enter_long_hedge_position(stock_1, stock_2, leverage)

        elif last_signal == -1:
            enter_short_hedge_position(stock_1, stock_2, leverage)

    count = 0
    while in_position():
        
        unrealised_abs = round(unrealised_profit(stock_1, stock_2)[0], 3)
        percent_profit = round(unrealised_profit(stock_1, stock_2)[1], 3)
        
        sys.stdout.write("\r")
        sys.stdout.write(str(count) + '_Current Profit: ' + str(unrealised_abs) + ' = '+ str(percent_profit) + '%')
        sys.stdout.flush()

        if percent_profit > 0.05:
            sys.stdout.write("\n")
            print('Take Profit Hit')
            take_profit(stock_1)
            take_profit(stock_2)
            pause_algo()

        if percent_profit < -0.025:
            sys.stdout.write("\n")
            print('Stop Loss Hit')
            take_profit(stock_1)
            take_profit(stock_2)
            pause_algo()

        count +=1
    
        if not in_position():
            execute_algo(stock_1, stock_2, leverage)


In [15]:
execute_algo('TECK', 'WPM', 5)



Assets chosen: TECK and WPM
Leverage: 5


ValueError: You must supply a method of authentication

In [ ]:
# dates_ran = []

# def check_date():
#     today = datetime.now()
#     dates_ran.append(today.strftime("%m/%d/%Y"))
#     time.sleep(10)
#     if len(dates_ran) < 100:
#         execute_algo()
#         return check_date()
    
# # check_date()

# 4. Visualize The Spread

In [ ]:
ax = downloaded_stockdata('TECK', 'WPM')[['TECK','WPM']].plot(figsize=(12, 6))#, title = 'Daily Closing Prices for {} and {}'.format(asset1,asset2))
ax.set_ylabel("Closing Price")
ax.grid(True);


In [ ]:
def spread_visualisation(stock_1, stock_2):
    selected_df = downloaded_stockdata(stock_1, stock_2) 
    plt.title(f'{stock_1}-{round(selected_df.hedge_ratio[-1],2)}*{stock_2}')
    downloaded_stockdata(stock_1, stock_2)['spread'].plot(figsize=(16,4), color='red')
    
spread_visualisation('TECK', 'WPM')

In [ ]:
def zscored_spread(stock_1, stock_2):
    downloaded_stockdata(stock_1, stock_2)['zscored'].plot(figsize=(16,4), color='orange')
    plt.title('Zscored Spread')
    plt.axhline(1, color='k')
    plt.axhline(-1, color='k')
    
zscored_spread('TECK', 'WPM')

In [ ]:
def visualise_returns(stock_1, stock_2, tp, sl):
    return_df = downloaded_stockdata(stock_1, stock_2).dropna()
    return_df['combined_return'] = return_df[f'{stock_1}_return'] + return_df[f'{stock_2}_return']
        #Trading Signal
    return_df['signal'] = return_df.apply(lambda x: 1 if ((x['zscored']<-1) or (x['combined_return']>tp))
                                              else (-1 if ((x['zscored']>1) or (x['combined_return']<sl)) else 0), axis=1)
    return_df['strategy_return'] = return_df[f'{stock_1}_forward_return']*return_df['signal'] +\
                                 return_df[f'{stock_2}_forward_return']*return_df['signal']*-return_df['hedge_ratio']

    portfolios_cumulative_return = np.exp(np.log1p(return_df['strategy_return']).cumsum())
    portfolios_cumulative_return.plot(figsize=(16,6), color='red')
    plt.title('Strategy Cumulative Returns')
    plt.ylabel('Return')
    plt.show()
    
visualise_returns('TECK', 'WPM', 0.05, -0.05)